# Structure of Transpiling

- There are four types of transpilers: Optimizer, Router, Unroller, and Placer.

- Transpilers within the Pass will execute sequentially.

- The Placer must run before the Router.

- The Placer should only be run once.


```
transpiler_list = [ ... ]
p = Passes(transpiler_list, connectivity, native gates)
transpiled_circ, final_layout = p(circuit)
```


## Passes

- A container for transpilers that need to be executed.

## Transpilers

### Optimizer

- `Preprocessing`: Matches the number of logical and physical qubits.
- `Rearrange`: Fuses gates.

### Placer

- `StarConnectivityPlacer`: Optimized qubit placement for the star layout.
- `Trivial`: Identity placer.
- `Custom`: Applies custom placement.
- `Subgraph`: Uses subgraph isomorphism.
- `Random`: Random placement.
- `ReverseTraversal`: Applies the SABRE algorithm on trivial placer.

### Router

 - `StarConnectivityRouter`: Greedily inserts swap gates for the star layout.

```py
# Pseudocode of finding the next middle qubit

def find(i, q1, q2):
    for j in range(i, len(gates)):
        if g[j] executes on q1 or q2:
            return q1 if g[j] executes on q1 else q2
        
        if not g[j] executes on q1 or q2:
            return q1

    return q1

for i in range(len(gates)):
    q1, q2 = g[i].qubits
    
    if q1 and q2 are not connected:    
        next_mid = find(i, q1, q2)
    
    move next_mid to the middle qubit
    q1 and q2 are connected
```
    
- `CircuitMap`
- `ShortestPaths`
- `Sabre`


### Unroller

- Consists of several decomposition algorithms.


# Usage

### Utils

#### `qibo.cskim_utils.circuit_info.count_czs_qibo`

```py
def count_czs_qibo(circuit):
    """
    Count the number of CZ gates in a Qibo circuit.
    """
    ...
```

#### `qibo.cskim_utils.connectivity.star_connectivity`

```py
def star_connectivity():
    """
    Returns a star graph with 5 nodes and 4 edges.
    """
    ...
```

#### `qibo.cskim_utils.gen_circuit.random_control_circuit_qibo`

```py
def random_control_circuit_qibo(n_qubits, n_cs, cx=False):
    """
    Generates a random control Qibo circuit with n_qubits qubits and n_cs control gates.
    If cx is True, the control gates are CX, otherwise they are CZ.
    """
```


### Transpilation

In [32]:
import networkx as nx

from qibo import gates
from qibo.models import Circuit
from qibo.transpiler.pipeline import Passes, assert_transpiling
from qibo.transpiler.optimizer import *
from qibo.transpiler.placer import *
from qibo.transpiler.router import *
from qibo.transpiler.unroller import *

import random

from qibo.cskim_utils.circuit_info import count_czs
from qibo.cskim_utils.connectivity import star_connectivity
from qibo.cskim_utils.gen_circuit import random_control_circuit_qibo

In [33]:
custom_passes = []
star_conn = star_connectivity()
default_gates = NativeGates.default()

# All Optimizer classes
opt_preprocessing = Preprocessing(connectivity=star_conn)
opt_rearrange = Rearrange()

# All Placer classes
plc_star = StarConnectivityPlacer()
plc_trivial = Trivial(connectivity=star_conn)
plc_subgraph = Subgraph(connectivity=star_conn)
plc_random = Random(connectivity=star_conn)

rt_star = StarConnectivityRouter()
plc_reversetrv = ReverseTraversal(connectivity=star_conn, routing_algorithm=rt_star)
# plc_cus = Custom(connectivity=star_conn)

# All Router classes
rt_star = StarConnectivityRouter()
rt_sabre = Sabre(connectivity=star_conn)
rt_shortestpth = ShortestPaths(connectivity=star_conn)
# rt_cm = CircuitMap(connectivity=star_conn)

# Unroller class
unr = Unroller(default_gates)

In [34]:
# Append transpiler classes and create a custom transpiler pipeline
custom_passes = []

# Optimizer
custom_passes.append(opt_preprocessing)

# Placer
custom_passes.append(plc_star)
# custom_passes.append(plc_trivial)
# custom_passes.append(plc_subgraph)
# custom_passes.append(plc_random)
# custom_passes.append(plc_reversetrv)
# custom_passes.append(plc_cus)

# Router
# custom_passes.append(rt_star)
custom_passes.append(rt_sabre)
# custom_passes.append(rt_shortestpth)
# custom_passes.append(rt_cm)

# Unroller
custom_passes.append(unr)

custom_pipeline = Passes(custom_passes, connectivity=star_conn, native_gates=default_gates)

In [35]:
# Generate a random circuit
circuit = random_control_circuit_qibo(5, 5, cx=False)
print("before transpilation: ")
print(circuit.draw())

# Transpile the circuit
transpiled_circ, final_layout = custom_pipeline(circuit)
print("\n\nafter transpilation: ")
print(transpiled_circ.draw(line_wrap=100))

before transpilation: 
q0: ───o─o─Z─o─
q1: ─Z─|─|─|─|─
q2: ─o─|─|─|─|─
q3: ───|─|─o─|─
q4: ───Z─Z───Z─


after transpilation: 
q0: ───────────────────────────────────────────o─
q1: ─────────Z─GPI2─Z─Z─GPI2─o─Z─GPI2─Z─Z─GPI2─|─
q2: ─o─o─Z─o────────o─Z─GPI2─Z─Z─GPI2─o────────Z─
q3: ─|─|─o─|─────────────────────────────────────
q4: ─Z─Z───Z─────────────────────────────────────
